In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
!pip install numpy requests nlpaug
!pip install torch>=1.6.0 transformers>=4.0.0 sentencepiece
!pip install simpletransformers>=0.61.10

     |████████████████████████████████| 405 kB 5.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.20 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.4.1 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.28.0 which is incompatible.


In [ ]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
aug = naw.ContextualWordEmbsAug(
    model_path='distilbert-base-uncased', action="insert",aug_max=10,device='cuda')

In [3]:
data=pd.read_csv('/content/drive/MyDrive/datatraine/new_train.csv')

In [4]:
data

,Unnamed: 0,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe,Labels,encoded_cat,clean_comment_text
0,0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,Normal,0,explanation edits made username hardcore metal...
1,1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,Normal,0,aww match background colour seemingly stuck th...
2,2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,Normal,0,hey man really trying edit war guy constantly ...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,Normal,0,make real suggestion improvement wondered sect...
4,4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,Normal,0,sir hero chance remember page
...,...,...,...,...,...,...,...,...,...,...,...
159566,159566,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,Normal,0,second time asking view completely contradicts...
159567,159567,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,Normal,0,ashamed horrible thing put talk page
159568,159568,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,Normal,0,spitzer umm there actual article prostitution ...
159569,159569,And it looks like it was actually you who put ...,0,0,0,0,0,0,Normal,0,look like actually put speedy first version de...


In [25]:
threat_data=data[data['loathe']==1]
threat_data

,Unnamed: 0,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe,Labels,encoded_cat,clean_comment_text
42,42.0,You are gay or antisemmitian? \n\nArchangel WH...,1.0,0.0,1.0,0,1.0,1.0,"malignant,rude,abuse,loathe",24.0,gay antisemmitian archangel white tiger meow g...
105,105.0,A pair of jew-hating weiner nazi schmucks.,1.0,0.0,1.0,0,1.0,1.0,"malignant,rude,abuse,loathe",24.0,pair jew hating weiner nazi schmuck
176,176.0,I think that your a Fagget get a oife and burn...,1.0,0.0,1.0,1,1.0,1.0,"malignant,rude,threat,abuse,loathe",28.0,think fagget get oife burn hell hate sorry can...
218,218.0,"Kill all niggers. \n\nI have hard, that others...",1.0,0.0,1.0,0,1.0,1.0,"malignant,rude,abuse,loathe",24.0,kill nigger hard others said included racist s...
238,238.0,u r a tw@ fuck off u gay boy.U r smelly.Fuck u...,1.0,0.0,1.0,0,1.0,1.0,"malignant,rude,abuse,loathe",24.0,u r tw fuck u gay boy u r smelly fuck ur mum p...
...,...,...,...,...,...,...,...,...,...,...,...
159281,159281.0,LoL!! \n\nyou're GAY!! you will never know how...,1.0,1.0,1.0,0,1.0,1.0,"malignant,highly_malignant,rude,abuse,loathe",13.0,lol gay never know good feel fuck woman as
159336,159336.0,"Oh, fuck off. The pansy Jew would just whine a...",1.0,0.0,1.0,0,1.0,1.0,"malignant,rude,abuse,loathe",24.0,oh fuck pansy jew would whine b nai brith beat...
159400,159400.0,"Shalom \n\nSemite, get the fuck out of here. I...",1.0,1.0,1.0,1,1.0,1.0,"malignant,highly_malignant,rude,threat,abuse,l...",17.0,shalom semite get fuck kill son bitch leave wi...
159449,159449.0,I think he is a gay fag!!!,1.0,0.0,0.0,0,0.0,1.0,"malignant,loathe",21.0,think gay fag


In [ ]:
def threat(df):
    new_text=[]
    
    ##selecting the minority class samples
    df_n=df[df.threat==1].reset_index(drop=True)

    ## data augmentation loop
    for i in tqdm_notebook(range(0,len(df_n))):
        
            text = df_n.iloc[i]['clean_comment_text']
            augmented_text = aug.augment(text,n=31)
            for i in augmented_text:
              new_text.append(i)
    
    
    ## dataframe
    new=pd.DataFrame({'clean_comment_text':new_text,'threat':1})
    frames=[df,new]
    df=pd.concat(frames)
    df=df.fillna(0)
    return df

In [5]:
! pip install sentence-splitter

     |████████████████████████████████| 44 kB 1.1 MB/s 


In [6]:
! pip install transformers

     |████████████████████████████████| 2.9 MB 2.7 MB/s 
     |████████████████████████████████| 3.3 MB 71.4 MB/s 
     |████████████████████████████████| 596 kB 85.0 MB/s 
     |████████████████████████████████| 56 kB 6.4 MB/s 
     |████████████████████████████████| 895 kB 82.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
! pip install SentencePiece

     |████████████████████████████████| 1.2 MB 4.2 MB/s 


In [8]:

import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [12]:
def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
df=threat_data['clean_comment_text'][0:20]

In [ ]:
for i in df:
  print(get_response(i,5))

In [21]:
def threat1(df):
    new_text=[]
    
    ##selecting the minority class samples
    df_n=df[df.threat==1].reset_index(drop=True)

    ## data augmentation loop
    for i in tqdm_notebook(range(0,len(df_n))):
        
            text = df_n.iloc[i]['clean_comment_text']
            augmented_text = get_response(text,30)
            for i in augmented_text:
              new_text.append(i)
    
    
    ## dataframe
    new=pd.DataFrame({'clean_comment_text':new_text,'threat':1})
    frames=[df,new]
    df=pd.concat(frames)
    df=df.fillna(0)
    return df

In [22]:
data=threat1(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/478 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3402: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [24]:
data

,Unnamed: 0,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe,Labels,encoded_cat,clean_comment_text
0,0.0,Explanation\nWhy the edits made under my usern...,0.0,0.0,0.0,0,0.0,0.0,Normal,0.0,explanation edits made username hardcore metal...
1,1.0,D'aww! He matches this background colour I'm s...,0.0,0.0,0.0,0,0.0,0.0,Normal,0.0,aww match background colour seemingly stuck th...
2,2.0,"Hey man, I'm really not trying to edit war. It...",0.0,0.0,0.0,0,0.0,0.0,Normal,0.0,hey man really trying edit war guy constantly ...
3,3.0,"""\nMore\nI can't make any real suggestions on ...",0.0,0.0,0.0,0,0.0,0.0,Normal,0.0,make real suggestion improvement wondered sect...
4,4.0,"You, sir, are my hero. Any chance you remember...",0.0,0.0,0.0,0,0.0,0.0,Normal,0.0,sir hero chance remember page
...,...,...,...,...,...,...,...,...,...,...,...
14335,0.0,0,0.0,0.0,0.0,1,0.0,0.0,0,0.0,"Tell anyone that rape jew family choke bagel, ..."
14336,0.0,0,0.0,0.0,0.0,1,0.0,0.0,0,0.0,"Tell anyone that rape jew family choke bagel, ..."
14337,0.0,0,0.0,0.0,0.0,1,0.0,0.0,0,0.0,"Tell anyone that rape jew family choke bagel, ..."
14338,0.0,0,0.0,0.0,0.0,1,0.0,0.0,0,0.0,Tell anyone that rape jew family choke bagel a...


In [23]:
data.to_csv("data_with_threat.csv")

In [10]:
def loathe1(df):
    new_text=[]
    
    ##selecting the minority class samples
    df_n=df[df.loathe==1].reset_index(drop=True)

    ## data augmentation loop
    for i in tqdm_notebook(range(0,len(df_n))):
        
            text = df_n.iloc[i]['clean_comment_text']
            augmented_text = get_response(text,10)
            for i in augmented_text:
              new_text.append(i)
    
    
    ## dataframe
    new=pd.DataFrame({'clean_comment_text':new_text,'loathe':1})
    frames=[df,new]
    df=pd.concat(frames)
    df=df.fillna(0)
    return df

In [11]:
loathe=loathe1(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1405 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3402: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [15]:
loathe

,Unnamed: 0,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe,Labels,encoded_cat,clean_comment_text
0,0.0,Explanation\nWhy the edits made under my usern...,0.0,0.0,0.0,0.0,0.0,0,Normal,0.0,explanation edits made username hardcore metal...
1,1.0,D'aww! He matches this background colour I'm s...,0.0,0.0,0.0,0.0,0.0,0,Normal,0.0,aww match background colour seemingly stuck th...
2,2.0,"Hey man, I'm really not trying to edit war. It...",0.0,0.0,0.0,0.0,0.0,0,Normal,0.0,hey man really trying edit war guy constantly ...
3,3.0,"""\nMore\nI can't make any real suggestions on ...",0.0,0.0,0.0,0.0,0.0,0,Normal,0.0,make real suggestion improvement wondered sect...
4,4.0,"You, sir, are my hero. Any chance you remember...",0.0,0.0,0.0,0.0,0.0,0,Normal,0.0,sir hero chance remember page
...,...,...,...,...,...,...,...,...,...,...,...
14045,0.0,0,0.0,0.0,0.0,0.0,0.0,1,0,0.0,The previous conversation was about liberal ea...
14046,0.0,0,0.0,0.0,0.0,0.0,0.0,1,0,0.0,The previous conversation was about eating lib...
14047,0.0,0,0.0,0.0,0.0,0.0,0.0,1,0,0.0,Mother always yelling dad beat shit and brothe...
14048,0.0,0,0.0,0.0,0.0,0.0,0.0,1,0,0.0,Mother always yelling dad beat shit and brothe...


In [16]:
loathe.to_csv("/content/drive/MyDrive/datatraine/loathe.csv")